In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.3.3.jar

--2022-03-07 02:28:29--  https://jdbc.postgresql.org/download/postgresql-42.3.3.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1039047 (1015K) [application/java-archive]
Saving to: ‘postgresql-42.3.3.jar.4’

postgresql-42.3.3.j 100%[===================>]   1015K  --.-KB/s    in 0.09s   

2022-03-07 02:28:29 (10.5 MB/s) - ‘postgresql-42.3.3.jar.4’ saved [1039047/1039047]



In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.3.3.jar").getOrCreate()

In [ ]:
#Get Data
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), inferSchema=True, sep="\t")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [ ]:
#Show schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
#drop null and duplicates
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

4863497
4863497
4863497


In [ ]:
#Data types check
rev_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [ ]:
#Change review date from string to data
from pyspark.sql.types import DateType
rev_df = rev_df.withColumn("review_date", rev_df["review_date"].cast(DateType()))

In [ ]:
#Establish Review DF
rev_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
rev_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10010KWXLSPF3|   43750961|B004UUHX9M|     247428181| 2014-12-05|
|R100BG8WSTAYPG|   37257103|B00ENKGLHO|       9108602| 2014-04-29|
|R100F00UQBXLAL|   14056891|B004O3CSX6|     579644974| 2011-11-11|
|R1010FUSHO8ASL|   30419449|B000VXNYQI|     496402705| 2012-12-17|
|R101LBRJ366XL1|   36812928|B000JFJC4G|     870047482| 2015-02-28|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
#Product dataframe
prod_df = df.select(["product_id", "product_title"])
prod_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B004UUHX9M|Star Wars Deluxe ...|
|B00ENKGLHO|Ducati 1199 Panig...|
|B004O3CSX6|Gund Peek A Boo Bear|
|B000VXNYQI|Rhode Island Nove...|
|B000JFJC4G|Toysmith Rainbow ...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
#prod count
count1 = prod_df.count()
count1

4863497

In [ ]:
#prod distinct
prod_df = prod_df.distinct()

count2 = prod_df.count()
count2

663991

In [ ]:
#customers dataframe
cust_df = df.groupBy("customer_id").count()

cust_df = cust_df.withColumnRenamed("count", "customer_count")

cust_df.orderBy(cust_df["customer_count"].desc()).show(10)


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   51591392|          1761|
|   49944666|           918|
|   16404530|           858|
|   22788662|           829|
|   52916805|           584|
|    7080939|           583|
|   25968347|           503|
|   15336438|           473|
|   48987445|           413|
|   41694086|           385|
+-----------+--------------+
only showing top 10 rows



In [ ]:
#Customer data check
cust_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
#change customer counter datatype
from pyspark.sql.types import IntegerType

cust_df = cust_df.withColumn("customer_count", cust_df["customer_count"].cast(IntegerType()))
# Verify Change 
cust_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [ ]:
#Establish Vine DF
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10010KWXLSPF3|          5|            0|          0|   N|
|R100BG8WSTAYPG|          5|            1|          1|   N|
|R100F00UQBXLAL|          5|            0|          0|   N|
|R1010FUSHO8ASL|          5|            0|          1|   N|
|R101LBRJ366XL1|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [ ]:
#Code to RDS
mode = "append"
jdbc_url="jdbc:postgresql://<MYENDPOINTFORDSWASHERE>"
config = {"user":"postgres", "password":"<MYPASSWORDWASHERE>", "driver" :"org.postgresql.Driver"}

In [ ]:
#write customer df to rds
cust_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [ ]:
#write review df to rds
rev_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [ ]:
#write products df to rds
prod_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [ ]:
#write vine df to rds
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)